# Assignment 9

Maksim Nikiforov

# Putting it all together
In this section we'll look at an example of fitting many models on a data set and choosing the overall best one via test error.

Process:
1. Read the data in (would then want to explore data, we'll skip this part)
2. Split the data into a training and test set
3. For each model type, select a **best** model using the training set (we'll use cross-validation but you could split the training set into a training and validation set instead)
4. Compare the best models on the test set.  Select the model with the lowest error (with considerations for simplicity)

## 1. Read in the `diamonds` data set
Comes from [kaggle](https://www.kaggle.com/datasets/shivam2503/diamondshttps://www.kaggle.com/datasets/shivam2503/diamonds).

In [75]:
import pandas as pd
import numpy as np
diamonds = pd.read_csv("data/diamonds.csv")

In [76]:
print(diamonds.columns)
diamonds.head()

# Notice an index column, "Unnamed: 0"
# We'll remove it and try to predict the price of our diamonds (our response variable)

Index(['Unnamed: 0', 'carat', 'cut', 'color', 'clarity', 'depth', 'table',
       'price', 'x', 'y', 'z'],
      dtype='object')


,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [77]:
# Drop first column
diamonds = diamonds.drop(diamonds.columns[0], axis = 1)
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Let's use $\$2401$, the median price of diamonds in this set, as the determinant for "expensive" and "normal" pricing.

In [79]:
# Find the median price of diamonds
diamonds["price"].median()

2401.0

Diamonds at or below this price point will belong to the "normal" (0) price category, and diamonds above $\$2401$ will be assigned to the "expensive" (1) category. Moreover, let's create dummy variables so we can include `cut` and `color`.  We'll remove the `clarity` variable and the continuous `price` variable and use `price_category` as our categorical response.

In [80]:
cut_dummies = pd.get_dummies(diamonds.cut)
color_dummies = pd.get_dummies(diamonds.color)
diamonds["price_category"] = 1
diamonds.loc[diamonds["price"] <= diamonds["price"].median(), "price_category"] = 0
diamonds = diamonds.drop(["clarity", "cut", "color", "price"], axis = 1)
diamonds = diamonds.join(cut_dummies).join(color_dummies)
diamonds.head()

,carat,depth,table,x,y,z,price_category,Fair,Good,Ideal,Premium,Very Good,D,E,F,G,H,I,J
0,0.23,61.5,55.0,3.95,3.98,2.43,0,0,0,1,0,0,0,1,0,0,0,0,0
1,0.21,59.8,61.0,3.89,3.84,2.31,0,0,0,0,1,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,4.05,4.07,2.31,0,0,1,0,0,0,0,1,0,0,0,0,0
3,0.29,62.4,58.0,4.20,4.23,2.63,0,0,0,0,1,0,0,0,0,0,0,1,0
4,0.31,63.3,58.0,4.34,4.35,2.75,0,0,1,0,0,0,0,0,0,0,0,0,1


Since we're using the median, we'll have roughly half of our diamonds in the "expensive" category and half in the "normal" category.

In [81]:
diamonds.price_category.value_counts()

0    26985
1    26955
Name: price_category, dtype: int64

Now let's check over the data to make sure the dummy variables aren't super rare.

In [ ]:
diamonds.describe()

,carat,depth,table,x,y,z,price_category,Fair,Good,Ideal,Premium,Very Good,D,E,F,G,H,I,J
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,5.731157,5.734526,3.538734,0.500204,0.029848,0.090953,0.399537,0.255673,0.223990,0.125603,0.181628,0.176900,0.209344,0.153949,0.100519,0.052058
std,0.474011,1.432621,2.234491,1.121761,1.142135,0.705699,0.500005,0.170169,0.287545,0.489808,0.436243,0.416919,0.331404,0.385541,0.381588,0.406844,0.360903,0.300694,0.222146
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.010000,79.000000,95.000000,10.740000,58.900000,31.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Note: ideally we would explore the data more and consider transformations of variables and other feature engineering.

## 2. Training and Test Split
First, let's just read in all the functions we'll need from `sklearn`.

In [83]:
# Import required libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

We can then separate our data into train (80%) and test (20%) sets. Our response column will be `price_category`, and the remaining columns will constitute our potential predictors.

In [85]:
# Create 80/20 split for our train and test data
X_train, X_test, y_train, y_test = train_test_split(
  diamonds.drop("price_category", axis = 1), # x variables (predictors)
  diamonds["price_category"],                # y variable (response)
  test_size=0.20, 
  random_state=42)

## 3. Fit and Select Models on Training Data

### Logistic Regression Models

Now, we can fit a series of logistic regression models. 

- `cv_full_model` uses the entire training set (all of our predictors)
- `cv_numeric_model` includes all of our numeric predictors (no dummy variables)
- `cv_dummy_model` uses only the dummy variables
- `cv_full_interaction_model` creates interaction terms but does not standardize variables
- `cv_numeric_interaction_model` - numeric-variable-only interaction model with categorical variables

We use 5-fold cross-validation and _accuracy_ as our loss function.

In [48]:
# Full model (entire training set)
cv_full_model = cross_validate(
    LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs"), 
    X = X_train, 
    y = y_train, 
    cv = 5)

# Numeric variables only (no dummy variables)
cv_numeric_model = cross_validate( 
    LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs"), 
    X = X_train[["carat", "depth", "table", "x", "y", "z"]], 
    y = y_train, 
    cv = 5)

# Use only dummy variables (columns 8 and beyond)
cv_dummy_model = cross_validate(     # only use our dummy variables
    LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs"), 
    X_train.iloc[:, 8:], 
    y_train, 
    cv = 5)

# Use PolynomialFeatures to create interaction terms, do not standardize variables first
poly = PolynomialFeatures(interaction_only=True, include_bias = False)  

# Full a full interaction model 
cv_full_interaction_model = cross_validate(
    LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs"), 
    poly.fit_transform(X_train), # fit all 1-way (first-order) interactions between our variables
    y_train, 
    cv = 5)

# Numeric-variable-only interaction model with categorical variables (columns 8 and beyond)
cv_numeric_interaction_model = cross_validate(
    LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs"), 
    np.concatenate((poly.fit_transform(X_train[["carat", "depth", "table", "x", "y", "z"]]), X_train.iloc[:, 8:].to_numpy()), axis = 1), 
    y_train, 
    cv = 5)

`cv_full_model`, `cv_numeric_model`, `cv_dummy_model`, `cv_full_interaction_model`, and `cv_numeric_interaction_model` all have the output of the `cross_validate` function for test scores. They were accuracy measures for each fold, and we want to combine them. We can do this by just finding the average.

In [50]:
print(cv_full_model['test_score'], 
      cv_numeric_model['test_score'], 
      cv_dummy_model['test_score'], 
      cv_full_interaction_model['test_score'], 
      cv_numeric_interaction_model['test_score']) 

[0.96176573 0.96025953 0.9603708  0.96280417 0.9612978 ] [0.94913683 0.94902097 0.94623407 0.95005794 0.95086906] [0.58382574 0.59309466 0.58667439 0.5801854  0.5783314 ] [0.96037539 0.95991195 0.96245655 0.96152955 0.96118192] [0.96199745 0.96025953 0.95886443 0.96257242 0.96095017]


In [92]:
# Find average accuracies for each model
[round(cv_full_model['test_score'].mean(),4), 
 round(cv_numeric_model['test_score'].mean(),4), 
 round(cv_dummy_model['test_score'].mean(),4),
 round(cv_full_interaction_model['test_score'].mean(),4), 
 round(cv_numeric_interaction_model['test_score'].mean(),4),
]

[0.9613, 0.9491, 0.5844, 0.9611, 0.9609]

`cv_full_model` appears to have the highest average accuracy at 0.9613, but it's nearly identical to the accuracy of `cv_full_interaction_model` and `cv_numeric_interaction_model`. We'll use `cv_full_model` as our "best" model, since it is the simplest. 

In [93]:
logistic_reg_best = LogisticRegression(penalty = "none", max_iter = 10000, solver = "lbfgs").fit(X_train, y_train)

### Classification Tree Model

We can use a `DecisionTreeClassifier` to fit classification trees, and use accuracy as our loss function.

Sometimes we want to fit a tree, but it ends up being overfitted. We can **prune** trees in a few ways, including cost-complexity pruning or cross-validation. We can also control the minimum number of samples that a leaf can have (`min_sample_leaf`), among other things.

The best combination can be determined using cross-validation, where we

1. Set up the values to consider (our tuning `parameters`)
2. Use `GridSearchCV()` to return the best values

In [57]:
parameters = {'max_depth': range(2,20), # how many splits we'll do
              'min_samples_leaf':[10, 50, 100, 250]}
tree_model = GridSearchCV(DecisionTreeClassifier(), # the model to use
                            parameters, 
                            cv = 5, 
                            scoring='accuracy') \
                          .fit(X_train, y_train)

The optimal combination of parameters appears to be `max_depth=7` and `min_samples_leaf=10`. 

In [58]:
print(tree_model.best_estimator_)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=10)


In [60]:
# best_score_ is accuracy for the "best" model
print(tree_model.best_score_, tree_model.best_params_)

0.9599554893413526 {'max_depth': 7, 'min_samples_leaf': 10}


We can use the `cross_validate` function to see the accuracies four our model using the optimal parameters.

In [61]:
ctree_cv = cross_validate(tree_model.best_estimator_,
                          X_train,
                          y_train,
                          cv = 5,
                          scoring='accuracy')

In [62]:
print(ctree_cv['test_score'])

[0.95968022 0.9609547  0.95793743 0.96141367 0.95979143]


We can save the model with the best estimator (`max_depth=7` and `min_samples_leaf=10`).

In [63]:
ctree_best = tree_model.best_estimator_.fit(X_train, y_train)

### Random Forest Model (Includes Bagged Tree as a Special Case)

Random forests use the same idea as bagging, but we don't use all predictors in each tree. Instead, we only use a random subset of predictors for each bootstrap sample/tree fit. Why? If we have a very strong predictor in our data set, every first split will split on that best variable. This makes bagged trees more correlated to one another. But using a random subset of predictors, we get less correlated trees, which in turn reduces variance.

In [66]:
parameters = {'max_depth': range(2,20), # how many splits we'll do
              'min_samples_leaf':[10, 50, 100, 250]}
rf_tune = GridSearchCV(RandomForestClassifier(),
                          parameters,
                          cv = 5) \
                          .fit(X_train, y_train)

In [67]:
print(rf_tune.best_estimator_)

RandomForestClassifier(max_depth=13, min_samples_leaf=10)


In [68]:
rf_cv = cross_validate(rf_tune.best_estimator_,
                       X_train,
                       y_train,
                       cv = 5)

In [69]:
print(rf_cv['test_score'])

[0.96083884 0.96141814 0.96071842 0.96373117 0.9617613 ]


In [70]:
rf_best = rf_tune.best_estimator_.fit(X_train, y_train)

## 4. Compare on the Test Set

We can compare the accuracy across all of our "best" models.

In [72]:
from sklearn.metrics import accuracy_score
logistic_pred = logistic_reg_best.predict(X_test)
ctree_pred = ctree_best.predict(X_test)
rf_pred = rf_best.predict(X_test)

print(accuracy_score(y_test, logistic_pred), accuracy_score(y_test, ctree_pred), accuracy_score(y_test, rf_pred))

0.9618094178717093 0.9591212458286985 0.9616240266963293


We get an accuracy in the range of 95.9%-96.2%. The logistic regression model is best with an accuracy of 96.18%, slightly beating our random forest model. 

## 5. Confusion matrix

In [73]:
# Create confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, rf_pred)
cm

array([[5160,  241],
       [ 173, 5214]])

The confusion matrix shows 5160 true negatives in the top left, 241 false positives in the top right, 173 false negatives in the lower left, and 5214 true positives in the lower right. 